tgb - 1/10/2022 - Testing whether adding tendencies from previous timesteps helps significantly improve performance

# Initialization

## Imports

In [1]:
from cbrain.climate_invariant import *
from cbrain.climate_invariant_utils import *

/nfspool-0/home/tbeucler/CBRAIN-CAM/notebooks/tbeucler_devlog


# Load dataset and define data generator

## Paths

In [121]:
path_data = '/DFS-L/DATA/pritchard/tbeucler/SPCAM/SPCAM_PHYS/'
in_vars = ['QBP','TBP','VBP','PS','SOLIN','SHFLX','LHFLX',
          'PHQt-dt','TPHYSTNDt-dt','FSNTt-dt',
           'FSNSt-dt','FLNTt-dt','FLNSt-dt', 'PRECTt-dt'] # We take the large-scale climate state as inputs
in_vars_ctrl = ['QBP','TBP','VBP','PS','SOLIN','SHFLX','LHFLX']
in_vars_check = ['QBP','TBP','VBP','PS','SOLIN','SHFLX','LHFLX',
                 'PHQ','TPHYSTND','FSNT',
                 'FSNS','FLNT','FLNS', 'PRECT'] # We take the large-scale climate state as inputs
out_vars = ['PHQ','TPHYSTND','FSNT','FSNS','FLNT','FLNS', 'PRECT'] # and we output the response of clouds/storms to these climate conditions
path_train = path_data + '2022_01_10_TRAIN_For_Nando_t-dt.nc'
path_valid = path_data + '2022_01_10_VALID_For_Nando_t-dt.nc'
path_input_norm = path_data + '2022_01_10_Norm_Outputs_t-dt.nc'

## Create new normalization file

In [30]:
norm_dataset = xr.open_dataset(path_input_norm)

In [31]:
norm_dataset

<xarray.Dataset>
Dimensions:           (var_names: 522, var_names_single: 29)
Coordinates:
  * var_names         (var_names) object 'QBP' 'QBP' ... 'Vdt_adiabatic'
  * var_names_single  (var_names_single) object 'QBP' 'QCBP' ... 'Vdt_adiabatic'
Data variables:
    mean              (var_names) float32 ...
    std               (var_names) float32 ...
    min               (var_names) float32 ...
    max               (var_names) float32 ...
    std_by_var        (var_names_single) float64 ...

In [32]:
var_names_full = norm_dataset['var_names'].values
var_names_full_single = norm_dataset['var_names_single'].values

In [33]:
for i in range(30):
    var_names_full = np.append(var_names_full,'PHQt-dt')
for i in range(30):
    var_names_full = np.append(var_names_full,'TPHYSTNDt-dt')
var_names_full = np.append(var_names_full,
                           ['FSNTt-dt','FSNSt-dt','FLNTt-dt','FLNSt-dt', 'PRECTt-dt'])

In [34]:
var_names_full_single = np.append(var_names_full_single,
                                 ['PHQt-dt','TPHYSTNDt-dt',
                                  'FSNTt-dt','FSNSt-dt','FLNTt-dt','FLNSt-dt', 'PRECTt-dt'])

In [35]:
new_coor = {}
new_coor['var_names'] = var_names_full
new_coor['var_names_single']= var_names_full_single

In [36]:
norm_data = {}

In [37]:
KEY = ['mean','std','min','max']

In [58]:
var_names_full[364:369]

array(['FSNT', 'FSNS', 'FLNT', 'FLNS', 'PRECT'], dtype=object)

In [59]:
for key in KEY:
    norm_data[key] = norm_dataset[key].values
    for i in range(30):
        norm_data[key] = np.append(norm_data[key],norm_data[key][154+i])
    for i in range(30):
        norm_data[key] = np.append(norm_data[key],norm_data[key][244+i])
    norm_data[key] = np.append(norm_data[key],norm_data[key][364:369])

In [62]:
key0 = 'std_by_var'

In [63]:
norm_data[key0] = norm_dataset[key0].values

In [79]:
var_names_full_single[np.concatenate(([9],[12],np.arange(16,21)))]

array(['PHQ', 'TPHYSTND', 'FSNT', 'FSNS', 'FLNT', 'FLNS', 'PRECT'],
      dtype=object)

In [80]:
norm_data[key0] = np.append(norm_data[key0],norm_data[key0][np.concatenate(([9],[12],np.arange(16,21)))])

In [83]:
norm_data_dict = {}

In [84]:
for key in KEY:
    norm_data_dict[key] = (['var_names'],norm_data[key])
norm_data_dict[key0] = (['var_names_single'],norm_data[key0])

In [85]:
new_norm = xr.Dataset(
    data_vars = norm_data_dict,
    coords = new_coor
)

In [87]:
path_save_norm = '/DFS-L/DATA/pritchard/tbeucler/SPCAM/SPCAM_PHYS/2022_01_10_Norm_Outputs_t-dt.nc'

In [88]:
new_norm.to_netcdf(path_save_norm,mode='w')

## Generators

In [90]:
scale_dict = pickle.load(open(path_data+'CIML_Zenodo/009_Wm2_scaling.pkl','rb'))

In [91]:
N_batch = 8192

train_gen_BF = DataGeneratorCI(
    data_fn = path_train,
    input_vars = in_vars,
    output_vars = out_vars,
    norm_fn = path_input_norm,
    input_transform = ('mean', 'maxrs'),
    output_transform = scale_dict,
    shuffle = False,
    batch_size=N_batch
)

In [92]:
valid_gen_BF = DataGeneratorCI(
    data_fn = path_valid,
    input_vars = in_vars,
    output_vars = out_vars,
    norm_fn = path_input_norm,
    input_transform = ('mean', 'maxrs'),
    output_transform = scale_dict,
    shuffle = False,
    batch_size=N_batch
)

In [114]:
train_gen_ctrl = DataGeneratorCI(
    data_fn = path_train,
    input_vars = in_vars_ctrl,
    output_vars = out_vars,
    norm_fn = path_input_norm,
    input_transform = ('mean', 'maxrs'),
    output_transform = scale_dict,
    shuffle = False,
    batch_size=N_batch
)

In [130]:
valid_gen_ctrl = DataGeneratorCI(
    data_fn = path_valid,
    input_vars = in_vars_ctrl,
    output_vars = out_vars,
    norm_fn = path_input_norm,
    input_transform = ('mean', 'maxrs'),
    output_transform = scale_dict,
    shuffle = False,
    batch_size=N_batch
)

In [122]:
train_gen_check = DataGeneratorCI(
    data_fn = path_train,
    input_vars = in_vars_check,
    output_vars = out_vars,
    norm_fn = path_input_norm,
    input_transform = ('mean', 'maxrs'),
    output_transform = scale_dict,
    shuffle = False,
    batch_size=N_batch
)

In [129]:
valid_gen_check = DataGeneratorCI(
    data_fn = path_valid,
    input_vars = in_vars_check,
    output_vars = out_vars,
    norm_fn = path_input_norm,
    input_transform = ('mean', 'maxrs'),
    output_transform = scale_dict,
    shuffle = False,
    batch_size=N_batch
)

In [102]:
train_gen_BF[0][0].shape

(8192, 159)

In [103]:
train_gen_BF[0][1].shape

(8192, 65)

# Neural networks

## With outputs[t-dt]

In [107]:
inp = Input(shape=(159,)) ## input after rh and tns transformation
densout = Dense(128, activation='linear')(inp)
densout = LeakyReLU(alpha=0.3)(densout)
for i in range (6):
    densout = Dense(128, activation='linear')(densout)
    densout = LeakyReLU(alpha=0.3)(densout)
dense_out = Dense(65, activation='linear')(densout)
model = tf.keras.models.Model(inp, dense_out)

In [108]:
model.compile(tf.keras.optimizers.Adam(), loss=mse)

In [109]:
# Where to save the model
path_HDF5 = '/DFS-L/DATA/pritchard/tbeucler/SPCAM/HDF5_DATA/'
save_name = '2022_01_10_Test_with_outputs_t-dt'

In [110]:
earlyStopping = EarlyStopping(monitor='val_loss', patience=10, verbose=0, mode='min')
mcp_save_pos = ModelCheckpoint(path_HDF5+save_name+'.hdf5',save_best_only=True, monitor='val_loss', mode='min')

In [112]:
Nep = 20
model.fit_generator(train_gen_BF, epochs=Nep, validation_data=valid_gen_BF,\
                    callbacks=[earlyStopping, mcp_save_pos])

Epoch 1/20
5759/5759 [==============================] - 704s 122ms/step - loss: 698.5109 - val_loss: 534.3392
Epoch 2/20
5759/5759 [==============================] - 620s 108ms/step - loss: 511.7525 - val_loss: 491.9944
Epoch 3/20
5759/5759 [==============================] - 359s 62ms/step - loss: 479.7723 - val_loss: 464.8938
Epoch 4/20
5759/5759 [==============================] - 390s 68ms/step - loss: 462.8350 - val_loss: 456.0069
Epoch 5/20
5759/5759 [==============================] - 381s 66ms/step - loss: 451.6360 - val_loss: 441.4028
Epoch 6/20
5759/5759 [==============================] - 384s 67ms/step - loss: 443.0424 - val_loss: 435.3585
Epoch 7/20
5759/5759 [==============================] - 418s 73ms/step - loss: 436.4950 - val_loss: 439.3909
Epoch 8/20
5759/5759 [==============================] - 365s 63ms/step - loss: 431.7736 - val_loss: 433.2342
Epoch 9/20
5759/5759 [==============================] - 377s 65ms/step - loss: 427.6234 - val_loss: 422.8021
Epoch 10/20
5759/

## Without outputs [t-dt] (ctrl)

In [116]:
inp = Input(shape=(94,)) ## input after rh and tns transformation
densout = Dense(128, activation='linear')(inp)
densout = LeakyReLU(alpha=0.3)(densout)
for i in range (6):
    densout = Dense(128, activation='linear')(densout)
    densout = LeakyReLU(alpha=0.3)(densout)
dense_out = Dense(65, activation='linear')(densout)
model2 = tf.keras.models.Model(inp, dense_out)

In [117]:
model2.compile(tf.keras.optimizers.Adam(), loss=mse)

In [118]:
# Where to save the model
path_HDF5 = '/DFS-L/DATA/pritchard/tbeucler/SPCAM/HDF5_DATA/'
save_name = '2022_01_10_Test_ctrl'

In [119]:
earlyStopping = EarlyStopping(monitor='val_loss', patience=10, verbose=0, mode='min')
mcp_save_pos = ModelCheckpoint(path_HDF5+save_name+'.hdf5',save_best_only=True, monitor='val_loss', mode='min')

In [120]:
Nep = 20
model2.fit_generator(train_gen_ctrl, epochs=Nep, validation_data=valid_gen_ctrl,\
                    callbacks=[earlyStopping, mcp_save_pos])

Epoch 1/20
5759/5759 [==============================] - 330s 57ms/step - loss: 1040.4823 - val_loss: 850.4765
Epoch 2/20
5759/5759 [==============================] - 349s 61ms/step - loss: 801.4659 - val_loss: 784.1218
Epoch 3/20
5759/5759 [==============================] - 319s 55ms/step - loss: 761.4127 - val_loss: 746.3516
Epoch 4/20
5759/5759 [==============================] - 351s 61ms/step - loss: 741.0682 - val_loss: 729.3660
Epoch 5/20
5759/5759 [==============================] - 396s 69ms/step - loss: 725.7501 - val_loss: 710.7130
Epoch 6/20
5759/5759 [==============================] - 335s 58ms/step - loss: 712.2777 - val_loss: 704.9843
Epoch 7/20
5759/5759 [==============================] - 346s 60ms/step - loss: 701.4451 - val_loss: 691.3455
Epoch 8/20
5759/5759 [==============================] - 355s 62ms/step - loss: 693.2357 - val_loss: 688.6079
Epoch 9/20
5759/5759 [==============================] - 361s 63ms/step - loss: 686.7506 - val_loss: 684.4446
Epoch 10/20
5759/5

## With outputs [t] (safety check that it performs better than with outputs [t-dt], because the NN has the answer in that case)

In [124]:
inp = Input(shape=(159,)) ## input after rh and tns transformation
densout = Dense(128, activation='linear')(inp)
densout = LeakyReLU(alpha=0.3)(densout)
for i in range (6):
    densout = Dense(128, activation='linear')(densout)
    densout = LeakyReLU(alpha=0.3)(densout)
dense_out = Dense(65, activation='linear')(densout)
model3 = tf.keras.models.Model(inp, dense_out)

In [125]:
model3.compile(tf.keras.optimizers.Adam(), loss=mse)

In [126]:
# Where to save the model
path_HDF5 = '/DFS-L/DATA/pritchard/tbeucler/SPCAM/HDF5_DATA/'
save_name = '2022_01_10_Test_check_with_answer'

In [127]:
earlyStopping = EarlyStopping(monitor='val_loss', patience=10, verbose=0, mode='min')
mcp_save_pos = ModelCheckpoint(path_HDF5+save_name+'.hdf5',save_best_only=True, monitor='val_loss', mode='min')

In [131]:
Nep = 20
model3.fit_generator(train_gen_check, epochs=Nep, validation_data=valid_gen_check,\
                    callbacks=[earlyStopping, mcp_save_pos])

Epoch 1/20
5759/5759 [==============================] - 676s 117ms/step - loss: 360.5617 - val_loss: 86.2019
Epoch 2/20
5759/5759 [==============================] - 617s 107ms/step - loss: 60.8476 - val_loss: 42.0848
Epoch 3/20
5759/5759 [==============================] - 474s 82ms/step - loss: 31.8291 - val_loss: 22.5468
Epoch 4/20
5759/5759 [==============================] - 465s 81ms/step - loss: 18.4504 - val_loss: 14.5692
Epoch 5/20
5759/5759 [==============================] - 472s 82ms/step - loss: 13.7249 - val_loss: 13.5438
Epoch 6/20
5758/5759 [============================>.] - ETA: 0s - loss: 11.2512

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



Traceback (most recent call last):
  File "/DFS-L/DATA/pritchard/tbeucler/Miniconda3_2021_04_12/miniconda3/envs/CbrainCustomLayer/lib/python3.6/site-packages/IPython/core/interactiveshell.py", line 3331, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "<ipython-input-131-aced3ca191ba>", line 3, in <module>
    callbacks=[earlyStopping, mcp_save_pos])
  File "/DFS-L/DATA/pritchard/tbeucler/Miniconda3_2021_04_12/miniconda3/envs/CbrainCustomLayer/lib/python3.6/site-packages/tensorflow_core/python/keras/engine/training.py", line 1297, in fit_generator
    steps_name='steps_per_epoch')
  File "/DFS-L/DATA/pritchard/tbeucler/Miniconda3_2021_04_12/miniconda3/envs/CbrainCustomLayer/lib/python3.6/site-packages/tensorflow_core/python/keras/engine/training_generator.py", line 323, in model_iteration
    steps_name='validation_steps')
  File "/DFS-L/DATA/pritchard/tbeucler/Miniconda3_2021_04_12/miniconda3/envs/CbrainCustomLayer/lib/python3.6/site-packages/tensorflow_core/p

KeyboardInterrupt: 

Safety check worked